**This notebook is an exercise in the [Time Series](https://www.kaggle.com/learn/time-series) course.  You can reference the tutorial at [this link](https://www.kaggle.com/ryanholbrook/linear-regression-with-time-series).**

---


In [ ]:
# Importamos librerias
from learntools.core import binder
binder.bind(globals())
from learntools.time_series.ex1 import *

from pathlib import Path
from learntools.time_series.style import * 

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [ ]:
# Obtenemos el Dataset [Este dataset registra el número de ventas de libros de tapa dura en una tienda minorista durante 30 días]
data_dir = Path('../input/ts-course-data/')
comp_dir = Path('../input/store-sales-time-series-forecasting')

book_sales = pd.read_csv(
    data_dir / 'book_sales.csv',
    index_col='Date',
    parse_dates=['Date'],
).drop('Paperback', axis=1)
book_sales['Time'] = np.arange(len(book_sales.index))
book_sales['Lag_1'] = book_sales['Hardcover'].shift(1)
book_sales = book_sales.reindex(columns=['Hardcover', 'Time', 'Lag_1'])

ar = pd.read_csv(data_dir / 'ar.csv')

dtype = {
    'store_nbr': 'category',
    'family': 'category',
    'sales': 'float32',
    'onpromotion': 'uint64',
}
store_sales = pd.read_csv(
    comp_dir / 'train.csv',
    dtype=dtype,
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales = store_sales.set_index('date').to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family'], append=True)
average_sales = store_sales.groupby('date').mean()['sales']

--------------------------------------------------------------------------------

# 1) Una de las ventajas de los modelos de regresión lineal es que son muy explicables, esto quiere decir que son faciles para interpretar sobre qué contribución hace cada caracteristica a las predicciones.

In [ ]:
# Graficamos regresión lineal obtenida paraa el dataset
fig, ax = plt.subplots()
ax.plot('Time', 'Hardcover', data=book_sales, color='0.75')
ax = sns.regplot(x='Time', y='Hardcover', data=book_sales, ci=None, scatter_kws=dict(color='0.25'))
ax.set_title('Gráfico de tiempo de ventas de libros de tapa dura');

Se plantea la incognita de ¿Cuanto en promedio se esperaria que cambiara las ventas de libros de tapa dura durante 6 dias? Para esto se hace un calculo simple dado a que el modelo de regresión lineal tiene una pendiente de 3.33, esto quiere decir que si multiplicamos este valor por los 6 dias obtendremos el resultado de ventas promedio de los libros de tapa dura.

In [ ]:
# Solución a la problemática
q_1.check()

-------------------------------------------------------------------------------

# 2) Las caracteristicas de retraso nos permiten modelar la dependencia de una serie. 

Una serie de tiempo tiene dependencia serial cuando una observación se puede predecir a partir de observaciones previas.

En el caso de nuestro dataset podemos predecir que las ventas altas en un dia generalmente significan ventas altas al dia siguiente.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(11, 5.5), sharex=True)
ax1.plot(ar['ar1'])
ax1.set_title('Series 1')
ax2.plot(ar['ar2'])
ax2.set_title('Series 2');

La gráfica de las "Series 1" tiene un peso de 0.95, debido a esto tenderá a tener valores con signos que permanecen iguales.
La gráfica de las "Series 2" tiene un peso de -0.95, debido a esto tenderá a tener valores con signos que oscilan entre valores negativos y positivos.

-------------------------------------------------------------------------------

# 3) Se crea un modelo de regresión lineal con una característica de paso de tiempo en la serie de ventas promedio de productos.

In [ ]:
from sklearn.linear_model import LinearRegression

df = average_sales.to_frame()

time = np.arange(len(df.index))

df['time'] = time 

X = df.loc[:, ['time']]  # Caracteristicas
y = df.loc[:, 'sales']   # Clase

# Se entrena el modelo
model = LinearRegression()
model.fit(X, y)

# Se almacena los valores ajustados como una serie de tiempo
y_pred = pd.Series(model.predict(X), index=X.index)

# Check your answer
q_3.check()

In [ ]:
# Se grafica el modelo de regresión lineal
ax = y.plot(**plot_params, alpha=0.5)
ax = y_pred.plot(ax=ax, linewidth=3)
ax.set_title('Gráfico de tiempo de ventas totales en tienda');

-------------------------------------------------------------------------------

# 4) Se crea un modelo de regresión lineal con una función de retraso en la serie de ventas promedio de productos.

In [ ]:
df = average_sales.to_frame()

lag_1 = df['sales'].shift(1)

df['lag_1'] = lag_1

X = df.loc[:, ['lag_1']]
X.dropna(inplace=True)  # Se eliminan los valores faltantes en el dataset de entrenamiento
y = df.loc[:, 'sales']  # Se crea el dataset de target
y, X = y.align(X, join='inner') 

model = LinearRegression() 
model.fit(X, y) # Se entrena el modelo de regresión lineal

# Predecimos y
y_pred = pd.Series(model.predict(X), index=X.index)

In [ ]:
# Graficamos el modelo obtenido
fig, ax = plt.subplots()
ax.plot(X['lag_1'], y, '.', color='0.25')
ax.plot(X['lag_1'], y_pred)
ax.set(aspect='equal', ylabel='Ventas', xlabel='Retraso_1', title='Gráfica de retraso de ventas promedio');